In [ ]:
# Copyright Simon Hanly-Jones, Emmanuel Isaac, Ryan Mann - simon.hanly.jones@gmail.com

#  This notebook is licenced under the Apache License 2.0 license.

Welcome to our DnD companion which takes a question about a monster, give you an answer, an image and a video of that monster.

Simon Hanly-Jones, Emmanuel Isaac, Ryan Mann

Copyright is asserted by the authors.

It can also be used for:
- prompt enhancement for image and video generation

- text to image and video generation

Please run everything, then use the GUI or run the test case by uncommenting demo_sequence() and running the cell


Colab Imports, run once

In [1]:
# may not need to install torch on colab
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

!pip install diffusers accelerate
!pip install langchain lancedb sentence_transformers
!pip install huggingface-hub
!pip install ctransformers[cuda]
!huggingface-cli download TheBloke/zephyr-7B-beta-GGUF zephyr-7b-beta.Q4_K_M.gguf --local-dir ./models --local-dir-use-symlinks False



running_on_intel = False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 22.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinu

Intel Imports - Uncomment entire block

In [ ]:
# !echo "Installation in progress..."
# !conda install -y --quiet  --prefix {sys.prefix} -c conda-forge \
#     accelerate==0.23.0 \
#     validators==0.22.0 \
#     diffusers==0.18.2 \
#     transformers==4.32.1 \
#     pillow \
#     ipywidgets \
#     ipython > /dev/null && echo "Installation successful" || echo "Installation failed"

# import sys
# !{sys.executable} -m pip install invisible-watermark --user > /dev/null 2>&1
# !{sys.executable} -m pip install transformers huggingface-hub --user > /dev/null 2>&1
# !{sys.executable} -m pip install langchain predictionguard lancedb sentence_transformers > /dev/null 2>&1
# !echo "Installation complete..."


Intel specific hardware function - run only on intel hardware

In [ ]:
# import random
# import requests
# import torch
# import torch.nn as nn
# import intel_extension_for_pytorch as ipex  # adds xpu namespace to PyTorch, enabling you to use Intel GPUs
# import validators
# import numpy as np

# def _optimize_pipeline(pipeline: DiffusionPipeline) -> DiffusionPipeline:
#     """
#     Optimizes the model for inference using ipex.

#     Parameters:
#     - pipeline: The model pipeline to be optimized.

#     Returns:
#     - pipeline: The optimized model pipeline.
#     """

#     for attr in dir(pipeline):
#         if isinstance(getattr(pipeline, attr), nn.Module):
#             setattr(
#                 pipeline,
#                 attr,
#                 ipex.optimize(
#                     getattr(pipeline, attr).eval(),
#                     dtype=pipeline.text_encoder.dtype,
#                     inplace=True,
#                 ),
#             )
#     return pipeline

# running_on_intel = True

Img to Video imports

In [2]:
import torch
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Text to Image imports

In [3]:
from diffusers import AutoPipelineForText2Image
import random

import torch
from PIL import Image, ImageTk
import datetime

Chatbot imports

In [4]:
import os
import json
import requests

from langchain.chains import LLMChain
from langchain.llms import PredictionGuard
from langchain.prompts import PromptTemplate
from langchain.document_loaders import JSONLoader

import lancedb
from lancedb.embeddings import with_embeddings
from sentence_transformers import SentenceTransformer

import pandas as pd
import ast



Evil Villian chatbot code. Also contains imports and generates the RAG database

Insallations for quantized zephyr

Create Quant Zephyr Instance courtesey of the Bloke

In [ ]:
# import ctransformers[cuda]

# # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
# llm = ctransformers.AutoModelForCausalLM.from_pretrained("TheBloke/zephyr-7B-alpha-GGUF", model_file="zephyr-7b-alpha.Q4_K_M.gguf", model_type="mistral", gpu_layers=50)

# print(llm("AI is going to"))

In [5]:
from langchain.llms import CTransformers
class Zephyr():
    def __init__(self):
        config = {
            "context_length" : 4096,
            "max_new_tokens" : 256,
            "temperature" : 0.7,
            "top_k" : 50 ,
            "top_p" : 0.95,
            "gpu_layers" : 50
        }

        self.llm = CTransformers(model='/content/models/zephyr-7b-beta.Q4_K_M.gguf', config=config)




In [6]:
class Monster_chatbot():
    def __init__(self, zephyr):




        self.embedding_model = SentenceTransformer("all-MiniLM-L12-v2")
        self.monster_table = self.init_monster_table(self.embedding_model)


        # Zephyr ‘syntax’

        # Real syntax used
        # Everything you see tagged with <|system|> is what the LLM expects as the main instruction, defining how it should approach user input. Everything tagged with <|user|> is what the LLM expects as user input, or even more broadly put, what it expects at “query time”. Finally, <|assistant|> tags the part of the prompt that the model will complete.
        self.info_context = self.rag_context_from_monster_table("aboleth")

        _monster_prompt_template = (
            "<|system|>You are a helpful assistant who is extremely knowledgeable about Dungeons and Dragons 5th Edition monsters. Your job is to give a comprehensive answer to the question from the provided context. Do so in the manner of an overacted, but honest villain.</s>\n"
            "<|user|>\n"
            "Context: {conversational_context}{info_context}\n"
            "Question: {query}\n"
            "</s>\n"
            "<|assistant|>\n"
        )


        self.messages = [
            {
                "user": "What is the armor class of an Aboleth?\n",
                "assistant": "The slippery Aboleth! Its feeble armor class is a mere 17, a laughable attempt at self-preservation through natural armor.\n"
            },
            {
                "user": "Is the Aboleth evil?\n",
                "assistant": "Evil, you ask? Chaotic evil courses through the veins of the wretched Aboleth! A malevolent force, craving nothing but chaos and destruction.\n"
            },
            {
                "user": "What kind of creature is a black dragon.\n",
                "assistant": "The black dragon! A colossal beast with scales as dark as the abyss. It commands fear, intelligence, and a loyalty that rivals even the most treacherous creatures. Beware its breath, a torrent of acid, a venomous cascade capable of melting steel!\n"
            },
            {
                "user": "Is it strong?\n",
                "assistant": "Strong? The Large Black Dragon is a behemoth! A strength of 23 and a strength modifier of +6. It scoffs at feeble attempts to challenge its might!\n"
            }
        ]


        monster_prompt = PromptTemplate(template=_monster_prompt_template, input_variables=["info_context", "conversational_context", "query"])


        self.monster_model = LLMChain(prompt=monster_prompt, llm=zephyr.llm, verbose=False)


    def get_last_monster(self):
        json_info =  ast.literal_eval(self.info_context)

        return json_info['name']
    def _update_info_context(self, new_info_entry):
        self.info_context = new_info_entry

    def _update_conversational_context(self, query, answer):
        # Update the conversational context with the new user query and assistant answer
        new_entry = {
                "user": f"{query}",
                "assistant": f"{answer}"
            }
        self.messages.append(new_entry)
        self.messages = self.messages[-8:]
        return



    def query(self, query):

        # print("\n\n\nQUESTION:", query)

        new_info_context = self.rag_context_from_monster_table(query)

        # print("RETRIEVED(first part of database record):",new_info_context[:30])

        self._update_info_context(new_info_context)
        answer = self.query_monster_model(query)
        if "\n" in answer:
            answer = answer.split('\n')[0]
        if "\\n" in answer:
            answer = answer.split('\\n')[0]

        answer = self.truncate_to_complete_sentence(answer)

        self._update_conversational_context(query, answer)

        # print("ANSWER:", answer )
        return answer

    def truncate_to_complete_sentence(self, answer):
        full_stop_index = 0
        for i, char in enumerate(answer):
            if char == "." or char == "?" or char == "!":
                full_stop_index = i

        answer = answer[: full_stop_index + 1]
        return answer


    def query_monster_model(self, query):


        answer = self.monster_model.run(conversational_context = self.messages,
                                            info_context = self.info_context,
                                            query = query,
                                            temperature=0.6,
                                            max_tokens=150,
                                            with_context = False)
        return answer

    def rag_context_from_monster_table(self, message):
        def embed(sentence):
            return self.embedding_model.encode(sentence)

        results = self.monster_table.search(embed(message)).limit(5).to_pandas()
        results.sort_values(by=["_distance"], inplace=True, ascending=True)
        return results["text"].values[0]


    def init_monster_table(self, embedding_model):
        # Creating lance database
        uri = "lancedb_challenge6"
        db = lancedb.connect(uri)

        try:
            # Try to open the table
            table = db.open_table("dnd_monsters")

        except Exception as e:
            # If the table doesn't exist, create and populate it
            # get .json database of DnD monsters, stored on google drive
            url = "https://drive.google.com/uc?id=14lZmwJBNiaozicvq8Prrm4GW3YKf5Ovn"
            response = requests.get(url)


            # Ensure the request was successful (status code 200)
            if response.status_code == 200:
                # Decode the content of the response as JSON
                monsters_data = json.loads(response.content)

                # If you want to save the content to a local file, you can do so:
                with open("monsters.json", "w") as file:
                    json.dump(monsters_data, file)

            else:
                print("Failed to retrieve the JSON file. Status code:", response.status_code)
                monsters_data = ''


            # Embedding data
            def embed_batch(batch):
                return [embedding_model.encode(sentence) for sentence in batch]

            metadata = []
            for i in range(len(monsters_data)):
                metadata.append([i, str(monsters_data[i])])
            doc_df = pd.DataFrame(metadata, columns=["chunk", "text"])
            embedded_data = with_embeddings(embed_batch, doc_df)

            # Create the table and add data
            db.create_table("dnd_monsters", data=embedded_data)
            table = db.open_table("dnd_monsters")

        return table


Text to image class uses sdxl-turbo to generate an image from the user's requested monster

In [7]:
class Text2ImageGeneratorSDXL:
    def __init__(self):
        # Initialize the text-to-image pipeline
        self.pipeline_text2image = AutoPipelineForText2Image.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16,
            variant="fp16")

        if running_on_intel:
            _optimize_pipeline(self.pipeline_text2image)
        else:
            self.pipeline_text2image = self.pipeline_text2image.to("cuda")
        # warmup
        self.pipeline_text2image(prompt="a castle", width=1024, height=576).images[0]
        self.pipeline_text2image(prompt="a wizard", width=1024, height=576).images[0]


    def generate(self, prompt, negative_prompt = "ugly, deformed, noisy, blurry, distorted, out of focus, bad anatomy, extra limbs"):

        # seed = 42
        seed = random.randint(-1000, 1000)
        generator = torch.manual_seed(seed)

        # # Generate and save the image
        image = self.pipeline_text2image(prompt=prompt, negative_prompt=negative_prompt,
                                          generator=generator, width=1024, height=576).images[0]

        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        filename = f"{timestamp}_image.png"
        image.save(filename)


        return image, filename

Image to Video Class using stable diffusion image to video on the generated image

In [8]:
class Image2VideoGenerator:
    def __init__(self):

        # Initialize the image-to-video pipeline

        # defualt setup
        self.pipe = StableVideoDiffusionPipeline.from_pretrained(
            "stabilityai/stable-video-diffusion-img2vid-xt", torch_dtype=torch.float16, variant="fp16"
        )

        if running_on_intel:
            _optimize_pipeline(self.pipe)
        else:
            self.pipe.enable_model_cpu_offload()

        # warmup
        image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/svd/rocket.png?download=true")
        image = image.resize((1024, 576))

        generator = torch.manual_seed(42)
        self.pipe(image, decode_chunk_size=8, generator=generator).frames[0]


    def resize_and_crop(self, image, target_size=(1024, 576)):
        # Open the image
        image

        # Check if the image size is different from the target size
        if image.size != target_size:
            # Resize the image to 1024x1024
            resized_image = image.resize((1024, 1024), Image.ANTIALIAS)

            # Crop the image to 1024x576 starting from the top
            left = 0
            top = 0
            right = target_size[0]
            bottom = target_size[1]

            cropped_image = resized_image.crop((left, top, right, bottom))

            return cropped_image
        else:
            # If the image is already the target size, return the original image
            return image


    def generate(self, image):
        image = self.resize_and_crop(image)

        # seed = 42
        seed = random.randint(-1000, 1000)
        generator = torch.manual_seed(seed)

        # regular usage
        frames = self.pipe(image, decode_chunk_size=8, generator=generator).frames[0]

        # extra motion setup
        # frames = pipe(image, decode_chunk_size=8, generator=generator, motion_bucket_id=180, noise_aug_strength=0.1).frames[0]

        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        filename = f"{timestamp}_video.mp4"

        export_to_video(frames, filename, fps=14)

        return filename


This is a prompt enhancement class to set up a good image to video result - Works with Zepher to inject some kind of appriopriate movement into the image, this is because the Image to Video performs much better with a clear action to animate. We don't want context to be saved as creatrues should be treated seperatly.

In [9]:
# Zephyr dynamic prompt enchancement agent using prediction guard chat interface
class PromptEnhancer_text2img():

    def __init__(self, zephyr):


        self.zephyr = zephyr
        template = """<|system|>
You are an AI language model that provides text-based responses to user inputs. Your responses should be relevant, accurate, and concise, and should reflect the capabilities and limitations of your programming. You should also strive to provide helpful and informative responses that add value to the user's experience.</s>
<|user|>
{question}</s>
<|assistant|>"""


        prompt = PromptTemplate(template=template, input_variables=["question"])

        self.prompt_model = LLMChain(prompt=prompt, llm=zephyr.llm)


        self.enchance_prompt("goblin")
        self.enchance_prompt("dragon")

    def enchance_prompt(self, monster):
        prompt =  f'improve this prompt for stable diffusion XL 1.0 base, incorporate an action or movement that would be appropriate for the given creature: "Concept art of a {monster} with a dramatic background, perfect anatomy, artstation, concept art, high definition, accent lighting" Respond only with improved prompts.'
        response = self.prompt_model.run(question = prompt, with_context = False)
        return response


This code is a wrapper to simply execution for the GUI

In [10]:
class Execution:
    def __init__(self):
        pass

    def get_video(self, image):
        image2video_generator = Image2VideoGenerator()
        video_filename = image2video_generator.generate(image)

        # cleanup
        del image2video_generator
        torch.cuda.empty_cache()

        return video_filename

    def get_image(self, prompt):
        text2image_generator = Text2ImageGeneratorSDXL()
        image, filename = text2image_generator.generate(prompt)

        # cleanup
        del text2image_generator
        torch.cuda.empty_cache()
        return image, filename

    def get_image_and_video(self, prompt):
        image, image_filename = self.get_image(prompt)
        video_filename = self.get_video(image)
        return image, image_filename, video_filename

    def submit_monster_question_get_answer_image_and_video(self, question):
        answer = 'not used'
        monster = 'not used'
        prompt = 'not used'
        image = 'not used'
        image_filename = 'not used'
        video_filename = 'not used'

        zephyr = Zephyr()
        monster_chatbot = Monster_chatbot(zephyr)

        answer = monster_chatbot.query(question)
        monster = monster_chatbot.get_last_monster()

        prompt_enhancer = PromptEnhancer_text2img(zephyr)
        prompt = prompt_enhancer.enchance_prompt(monster)

        del zephyr
        del prompt_enhancer
        del monster_chatbot
        torch.cuda.empty_cache()



        image, image_filename, video_filename = self.get_image_and_video(prompt)
        return answer, monster, prompt, image, image_filename, video_filename


    def submit_generic_prompt_get_image_and_video(self, prompt):
        answer = 'not used'
        monster = 'not used'
        image = 'not used'
        image_filename = 'not used'
        video_filename = 'not used'

        image, image_filename, video_filename = self.get_image_and_video(prompt)


        return answer, monster, prompt, image, image_filename, video_filename

    def submit_any_creature_get_image_and_video(self, creature):
        answer = 'not used'
        monster = 'not used'
        prompt = 'not used'
        image = 'not used'
        image_filename = 'not used'
        video_filename = 'not used'

        zephyr = Zephyr()
        prompt_enhancer = PromptEnhancer_text2img(zephyr)
        prompt = prompt_enhancer.enchance_prompt(creature)

        del zephyr
        del prompt_enhancer
        torch.cuda.empty_cache()

        image, image_filename, video_filename = self.get_image_and_video(prompt)
        return answer, monster, prompt, image, image_filename, video_filename

    def submit_monster_question_get_answer(self, question):
        zephyr = Zephyr()
        monster_chatbot = Monster_chatbot(zephyr)
        answer = monster_chatbot.query(question)
        del zephyr
        del monster_chatbot
        torch.cuda.empty_cache()

        return answer


Code to make the GUI

In [11]:
import ipywidgets as widgets
from IPython.display import display, clear_output

class ExecutionGUI:
    def __init__(self, execution):
        self.execution = execution

        # Create buttons with larger style
        button_style = {'description_width': 'initial'}
        self.button_question = widgets.Button(description="Submit DnD Monster Monster Question - Full Pipeline - INPUT: Question GET: Answer - Prompt Enhance - Image - Video", style=button_style, layout=widgets.Layout(width='auto'))
        self.button_generic_prompt = widgets.Button(description="Submit Generic Prompt (eg normal sdxl prompt) - INPUT: Prompt (not enhanced) GET: Image - Video", style=button_style, layout=widgets.Layout(width='auto'))
        self.button_any_creature = widgets.Button(description="Submit Any Creature (eg 'frog') - INPUT:creature GET: Prompt Enhance - Image - Video", style=button_style, layout=widgets.Layout(width='auto'))
        self.button_monster_question_get_answer = widgets.Button(description="Submit DnD Monster Question ('Is a Griffon fast?'') (Answer Only)", style=button_style, layout=widgets.Layout(width='auto'))

        # Create input widgets with larger style
        input_style = {'description_width': 'initial'}
        self.text_question = widgets.Text(description="Question:", style=input_style, layout=widgets.Layout(width='auto'))
        self.text_generic_prompt = widgets.Text(description="Generic Prompt:", style=input_style, layout=widgets.Layout(width='auto'))
        self.text_any_creature = widgets.Text(description="Creature:", style=input_style, layout=widgets.Layout(width='auto'))
        self.text_monster_question_get_answer = widgets.Text(description="Monster Question:", style=input_style, layout=widgets.Layout(width='auto'))

        # Output widget
        self.output = widgets.Output()

        # Assign functions to buttons
        self.button_question.on_click(self.submit_monster_question)
        self.button_generic_prompt.on_click(self.submit_generic_prompt)
        self.button_any_creature.on_click(self.submit_any_creature)
        self.button_monster_question_get_answer.on_click(self.submit_monster_question_get_answer)

        # Display widgets
        display(self.text_question, self.button_question, display_id='output1')
        display(self.text_generic_prompt, self.button_generic_prompt, display_id='output2')
        display(self.text_any_creature, self.button_any_creature, display_id='output3')
        display(self.text_monster_question_get_answer, self.button_monster_question_get_answer, display_id='output4')
        display(self.output, display_id='output5')

    def submit_monster_question(self, _):
        question = self.text_question.value
        with self.output:
            self.output.clear_output(wait=True)
            result = self.execution.submit_monster_question_get_answer_image_and_video(question)
            self.display_result(result)

    def submit_generic_prompt(self, _):
        prompt = self.text_generic_prompt.value
        with self.output:
            self.output.clear_output(wait=True)
            result = self.execution.submit_generic_prompt_get_image_and_video(prompt)
            self.display_result(result)

    def submit_any_creature(self, _):
        creature = self.text_any_creature.value
        with self.output:
            self.output.clear_output(wait=True)
            result = self.execution.submit_any_creature_get_image_and_video(creature)
            self.display_result(result)

    def submit_monster_question_get_answer(self, _):
        question = self.text_monster_question_get_answer.value
        with self.output:
            self.output.clear_output(wait=True)
            result = self.execution.submit_monster_question_get_answer(question)
            print(f"Answer: {result}")

    def display_result(self, result):
        print("\n\n\nRESULTS:\n")
        answer, monster, prompt, image, image_filename, video_filename = result
        print(f"Answer to your question: {answer}\n")
        print(f"Monster identified as: {monster}\n")
        print(f"Prompt enhanced to: {prompt}\n")
        print(f"Image: {image_filename}\n")
        display(image)
        print(f"\nVideo: {video_filename}\n")



This is function runs the demo sequence, it takes about 30 mins on a A100 or v100 GPU.

In [12]:
from IPython.display import display
from IPython.display import Video

def demo_sequence():

    questions = [
        "Tell me about Medusa's attacks.",
        "Is a Griffon fast?",
        "What is the speed of a Blink Dog?",
        "What is the armor class of an Bugbear?",
        "Is a Goblin sneaky?",
        "Is a Giant Toad perceptive?",
        "Is a Tarrasque strong?",
        "Tell me about a Chimera's strength.",
        "Is a Young Red Dragon evil?",
        "Is a Sea Hag fast?",
        "How smart is a Kobold?",
        "What is an Owlbear?",
        "What languages does Pegasus know?",
        "What is the challenge rating of an Ancient Blue Dragon?",
        "What is the armor class of an Aboleth?",
    ]

    zephyr = Zephyr()
    monster_chat = Monster_chatbot(zephyr)
    answers = []
    monsters = []
    for question in questions:
        answer = monster_chat.query(question)
        answers.append(answer)
        monsters.append(monster_chat.get_last_monster())


    prompts = []

    prompt_enhancer = PromptEnhancer_text2img(zephyr)
    for monster in monsters:
        monster = monster.lower()
        prompt = prompt_enhancer.enchance_prompt(monster)
        prompts.append(prompt)

    del monster_chat
    del zephyr
    del prompt_enhancer
    torch.cuda.empty_cache()

    imgs = []
    image_filenames = []


    image_generator = Text2ImageGeneratorSDXL()
    for prompt in prompts:
        img, image_filename = image_generator.generate(prompt=prompt)
        imgs.append(img)
        image_filenames.append(image_filename)

    del image_generator
    torch.cuda.empty_cache()

    video_generator = Image2VideoGenerator()

    video_paths = []
    for img in imgs:

        video_path = video_generator.generate(img)
        video_paths.append(video_path)

    del video_generator
    torch.cuda.empty_cache()

    for question, answer, monster, prompt, img, image_filename, video_path in zip(questions, answers, monsters, prompts, imgs, image_filenames, video_paths):

        print(f"Step 1 Question - The User Asks:\n     {question}\n")
        print(f"Step 2 Monster - The question is queried against the RAG database with Sentence Transformer all-MiniLM-L12-v2. The record retrieved and injected is:\n     {monster}\n")
        print(f"Step 3 Answer - The evil Zephyr Bot receives a RAG injection and responds:\n     {answer}\n")
        print(f"Step 4 Prompt - The subject monster is acquired from the RAG response and turned into a dynamic Text2Img prompt with a separate Zephyr ChatBot:\n     {prompt}\n")
        print(f"Step 5 Image - The prompt is turned into an image with SDXL. The image path is:\n     {image_filename}\n")
        display(img)
        print(f"Step 6 Video - The image is turned into a video with Stable Diffusion Img2Vid-XT. The video path is:\n     {video_path}\n\n")


    # save demo text to file

    output_filename = 'demo_text.txt'

    with open(output_filename, 'w') as output_file:
        for question, answer, monster, prompt, img, image_filename, video_path in zip(questions, answers, monsters, prompts, imgs, image_filenames, video_paths):
            output_file.write(f"Step 1 Question - The User Asks:\n     {question}\n\n")
            output_file.write(f"Step 2 Monster - The question is queried against the RAG database with Sentence Transformer all-MiniLM-L12-v2. The record retrieved and injected is:\n     {monster}\n\n")
            output_file.write(f"Step 3 Answer - The evil Zephyr Bot receives a RAG injection and responds:\n     {answer}\n\n")
            output_file.write(f"Step 4 Prompt - The subject monster is acquired from the RAG response and turned into a dynamic Text2Img prompt with a separate Zephyr ChatBot:\n     {prompt}\n\n")
            output_file.write(f"Step 5 Image - The prompt is turned into an image with SDXL. The image path is:\n     {image_filename}\n\n")
            # display(img)
            output_file.write(f"Step 6 Video - The image is turned into a video with Stable Diffusion Img2Vid-XT. The video path is:\n     {video_path}\n\n")
            # play_video(video_path)

    return



The GUI, hopefully the buttons are self-explanatory. Please note execution takes a long time,

YOU CANNOT RUN THE GUI AND THE DEMO SEQUENCE AT THE SAME TIME

In [13]:
# Create an instance of Execution and ExecutionGUI
execution = Execution()
gui = ExecutionGUI(execution)


Text(value='', description='Question:', layout=Layout(width='auto'), style=DescriptionStyle(description_width=…

Button(description='Submit DnD Monster Monster Question - Full Pipeline - INPUT: Question GET: Answer - Prompt…

Text(value='', description='Generic Prompt:', layout=Layout(width='auto'), style=DescriptionStyle(description_…

Button(description='Submit Generic Prompt (eg normal sdxl prompt) - INPUT: Prompt (not enhanced) GET: Image - …

Text(value='', description='Creature:', layout=Layout(width='auto'), style=DescriptionStyle(description_width=…

Button(description="Submit Any Creature (eg 'frog') - INPUT:creature GET: Prompt Enhance - Image - Video", lay…

Text(value='', description='Monster Question:', layout=Layout(width='auto'), style=DescriptionStyle(descriptio…

Button(description="Submit DnD Monster Question ('Is a Griffon fast?'') (Answer Only)", layout=Layout(width='a…

Output()

To run the demo_sequence uncomment and run this.

YOU CANNOT RUN THE GUI AND THE DEMO SEQUENCE AT THE SAME TIME - PLEASE COMMENT OUT, OR DON'T RUN, THE ABOVE CELL IF YOU WANT TO RUN THE DEMO

The images and videos are saved to files and the explaination text is written to 'demo_text.txt'. It will generate 15 exmaples of the full pipeline

In [ ]:
# demo_sequence()

Code to download the content folder of the colab - only run to zip and download everything. If you want to save your images and video, uncomment and run

In [ ]:
# import os
# from google.colab import files

# def save_content_files(zip_filename='content_data.zip'):
#     """
#     Save all files (excluding subfolders) in the /content folder to a zip file and download it.

#     Parameters:
#     - zip_filename (str): Name of the zip file to be created.

#     Returns:
#     - None
#     """
#     # List all files in the /content folder (excluding subfolders)
#     files_to_zip = [f for f in os.listdir('/content') if os.path.isfile(os.path.join('/content', f))]

#     # Create a zip archive of the selected files
#     os.system(f"zip -r {zip_filename} {' '.join(files_to_zip)}")

#     # Download the zip file
#     files.download(f'/content/{zip_filename}')

# # Example usage:
# # Specify the name of the zip file (optional)
# save_content_files(zip_filename='content_data.zip')

In [ ]:
# @misc{tunstall2023zephyr,
#       title={Zephyr: Direct Distillation of LM Alignment},
#       author={Lewis Tunstall and Edward Beeching and Nathan Lambert and Nazneen Rajani and Kashif Rasul and Younes Belkada and Shengyi Huang and Leandro von Werra and Clémentine Fourrier and Nathan Habib and Nathan Sarrazin and Omar Sanseviero and Alexander M. Rush and Thomas Wolf},
#       year={2023},
#       eprint={2310.16944},
#       archivePrefix={arXiv},
#       primaryClass={cs.LG}}